In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parents[1]))
from model import left_right_task as lrt, network_model, util, plot_style

# Load the fiducial network
Wji, pset, amp, dur, l_kernel, r_kernel = util.load_fiducial_network(True)

Wji_means = np.array([np.mean(w[np.eye(w.shape[0], dtype=bool) == False]) for w in Wji])


In [5]:
pset

array([ 1.00000000e-02,  1.00000000e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.19792554e+01,  4.36437108e+01, -4.75062770e+00, -1.03428554e+01,
        7.39000000e+00,  1.04790000e+02])

In [6]:
data_dir = 'figures/figure8/data'
output_dir = util.make_data_folder(data_dir, name='raw_reliabilities')
CVs = np.logspace(-2, 0, 5)

TypeError: make_data_folder() got an unexpected keyword argument 'name'

In [3]:
# task parameters
numPairs = 5
dt = 1e-5
seq_len = 6
sequences = lrt.make_all_sequences(seq_len, ['L', 'R'])
equil_duration = 2

In [ ]:
# universal parameter sweep
n = 25
amp_range = np.logspace(np.log10(7.39), 2, n) # 7.39 is thetaE -- need to turn it on to get a response
dur_range = np.linspace(0, 0.05, n)
dur_mesh, amp_mesh = np.meshgrid(dur_range, amp_range) # amp on y-axis, duration on x-axis
dur_flat, amp_flat = dur_mesh.ravel(), amp_mesh.ravel()

In [ ]:
def run_model(Wji, dur, amp):
    if dur == 0:
        return 0.5  # return a default reliability for zero duration

    # define the stimulus
    stim_map = lrt.make_stim_map(numPairs, amp, dur, l_kernel, r_kernel, dt)

    # run the model
    FSM = lrt.make_FSM(numPairs, pset, Wji, stim_map, 2, dt=dt)
    reliability = lrt.FSM_reliability(sequences, FSM)
    return reliability


In [ ]:
def run_sweep(CV, i):
    # make the Wji matrix
    Wji = util.make_Wji(Wji_means, CV)

    # run the model for each combination of duration and amplitude
    reliabilities = np.array([run_model(Wji, dur, amp) for dur, amp in zip(dur_flat, amp_flat)])

    # reshape the results to match the input grid
    reliability_grid = reliabilities.reshape(dur_mesh.shape)

    # save the results
    